ETL

Se realiza la carga y transformaciones necesarias de los archivos 
- 'australian_user_reviews.json'
- 'australian_users_items.json'
- 'output_steam_games.json'

De estas tranformaciones se exportan los archivos csv: 'd_games.csv', 'd_item_user.csv' y 'd_reviuws.csv' para su posterior desarrollo de funcions de consultas del proyecto.

In [24]:
# Se Importan las librerias requeridas
import pandas as pd
import json
import ast
from textblob import TextBlob
import re




1. DATA STEAM GAME

In [2]:
# Se abre el archivo en formato json "steam_games" y se lee el archivo linea por linea y aplicamos la funcion json.loads
# a cada una de las lineas
data = []
with open('output_steam_games.json') as steam:
    for linea in steam:
        dato = json.loads(linea)
        data.append(dato) 

In [59]:
# Convertimos nuestra data a un DataFrame para que nos facilite el trabajo transformacion de la data

steam_df = pd.DataFrame(data)
steam_df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561197970982479,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,js41637,76561198035864385,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,evcentric,76561198007712555,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Riot-Punch,76561197963445855,"[{'item_id': '10', 'item_name': 'Counter-Strik...",328.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,doctr,76561198002099482,"[{'item_id': '300', 'item_name': 'Day of Defea...",541.0


In [60]:
# Se realiza un checkeo general del dataframe y asi tener mejor comprension de los campos que contiene
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  object 
 12  id              32133 non-null  object 
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
 15  user_id         88310 non-null  object 
 16  steam_id        88310 non-null  object 
 17  items           88310 non-nul

In [61]:
# Despues de mirar cada uno de los campos se eliminan los inecesarios y se consideran campos especificos,
# los cuales son requeridos para el desarrollo de las funciones
steam_df.drop(columns=['title', 'user_id', 'steam_id', 'items', 'items_count', 'discount_price', 'metascore','tags', 'specs', 'url','reviews_url'], inplace=True)


In [62]:
# Al contener demasiados nulos se toma la accion de eliminar y ver como queda compuesto el dataframe
steam_df.dropna(how='all', inplace=True)


In [63]:
steam_df.shape

(32135, 8)

In [64]:
# Obtenemos informacion del nuevo dataframe
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   release_date  30068 non-null  object
 4   price         30758 non-null  object
 5   early_access  32135 non-null  object
 6   id            32133 non-null  object
 7   developer     28836 non-null  object
dtypes: object(8)
memory usage: 2.2+ MB


In [65]:
# Checkeamos campos especificos del dataframe, en este caso 'genres'
steam_df['genres'].explode().unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', nan, 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [66]:
# Ahora en el campo 'release_date', se realiza varias transformaciones como:
# Se guardan solo los registron que no tengan release_date nula 
steam_df = steam_df[steam_df['release_date'].notna()]


In [67]:
# Se hace el cambio de formato de fecha a AAAA-mm-dd 
steam_df['release_date']= pd.to_datetime(steam_df['release_date'],format='%Y-%m-%d', errors='coerce')


In [68]:
# Se crea una nueva columna 'release_year', donde se extrae el año de especifico de lanzamiento
steam_df['year'] = steam_df['release_date'].dt.year
steam_df['year']= steam_df['year'].astype('Int16')


In [69]:
# Como se puede observar el campo 'price' es de tipo object, contiene registro no numericos 
steam_df['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 3.99, 9.99, 18.99,
       29.99, nan, 10.99, 2.99, 1.59, 14.99, 1.99, 59.99, 8.99, 6.99,
       7.99, 39.99, 'Free', 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99,
       79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99, 149.99,
       771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99, 179.0,
       26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 22.99, 320.0, 38.85,
       71.7, 995.0, 27.49, 3.39, 6.0, 19.95, 499.99, 199.99, 16.06, 4.68,
       131.4, 44.98, 202.76, 

In [76]:
# Remmplazamos para extraer el valor especifico
steam_df['price'] = steam_df['price'].replace('Starting at $499.00', '499.0')
steam_df['price'] = steam_df['price'].replace('Starting at $499.00', '499.0')


In [81]:
# Se convierte el campo en numerico y rellenamos los no numericos en 0
steam_df['price'] = pd.to_numeric(steam_df['price'], errors='coerce').fillna(0)

In [84]:
#Cambiamos el formato
steam_df['price'] = steam_df['price'].astype(float) 

In [49]:
#En el campo 'id', borramos nulos y covertimos a int
steam_df = steam_df.dropna(subset=['id'])


In [50]:
steam_df['id'] = steam_df['id'].astype(int)


In [57]:
steam_df['publisher'].count()

30067

In [58]:
# Se rellenan los nulos de los campos 'publisher','app_name' y 'developer', por 'no info'
steam_df['publisher'] = steam_df['publisher'].fillna('no info')
steam_df['publisher'] = steam_df['publisher'].fillna('no info')
steam_df['publisher'] = steam_df['publisher'].fillna('no info')


In [ ]:
steam_df.info()

In [ ]:
steam_df.to_csv('d_games.csv')

2. DATASET USER ITEMS


In [3]:
data1 = []
with open('australian_users_items.json',encoding='utf-8') as item:
    for linea in item.readlines():
        data1.append(ast.literal_eval(linea))

In [4]:
#Convertimos a Dataframe
item_df = pd.DataFrame(data1)

In [5]:
item_df.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [6]:
# En el campo 'item' se encuentran listas de diccionarios, se utiliza la funcion 'json_normsliza' para desanidar
df_items2 = pd.json_normalize(data1, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )


In [ ]:
# Observamos el dataframe
df_items2.describe()

In [10]:
# Se eliminan duplicados
df_items2 = df_items2.drop_duplicates(keep='first')


In [7]:
# Se elimina el campo playtime_2weeks 
df_items2 = df_items2.drop('playtime_2weeks', axis=1)


In [9]:
#Revisamos su composicion
df_items2.columns

Index(['item_id', 'item_name', 'playtime_forever', 'steam_id', 'items_count',
       'user_id', 'user_url'],
      dtype='object')

In [13]:
# Se crea el archivo csv 'd_item_user'
df_items2.to_csv('d_item_user.csv')

3. DATA USER REVIEW

In [3]:
# Se abre el archivo 'australian_user_reviews'
data2 = []
with open('australian_user_reviews.json',encoding='utf-8') as reviuw:
    for linea in reviuw.readlines():
        data2.append(ast.literal_eval(linea))

In [5]:
#Convertimos a Dataframe
reviuw_df = pd.DataFrame(data2)

In [6]:
# Se checkea como esta compuesto, y se observa que campo 'reviews' esta anidado
reviuw_df.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [9]:
# Por medio de este For, se puede acceder a los campos dentro del campo reviews y obtener un nuevo Dataframe
data_des = []
for i, e in reviuw_df.iterrows():
    user_id = e['user_id']
    user_url = e['user_url']
    for reseñas in e['reviews']:
        reseñas['user_id'] = user_id
        reseñas['user_url'] = user_url
        data_des.append(reseñas)

df_reviuw = pd.DataFrame(data_des)

In [10]:
df_reviuw.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637


In [12]:
df_reviuw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        59305 non-null  object
 1   posted       59305 non-null  object
 2   last_edited  59305 non-null  object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  bool  
 6   review       59305 non-null  object
 7   user_id      59305 non-null  object
 8   user_url     59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


In [18]:
# Se crea la funcion sentimiento para  aplicar análisis de sentimiento con NLP 
# debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo
def sentiment(data):
    npl = TextBlob(data)
    if npl.sentiment.polarity < 0:
        return 0
    elif npl.sentiment.polarity == 0:
        return 1
    else:
        return 2

In [19]:
df_reviuw['sentiment_analisy']=df_reviuw['review'].apply(sentiment)

In [20]:
df_reviuw.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url,sentiment_analisy
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,2
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,0


In [21]:
# Se elimina el campo 'review'
df_reviuw = df_reviuw.drop('review', axis=1)


In [22]:
df_reviuw.head()

,funny,posted,last_edited,item_id,helpful,recommend,user_id,user_url,sentiment_analisy
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,2
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,0


In [27]:
#Se crea una funcion para cambiar el formato del campo posted a formato "YYYY-MM-DD"
def convertir_fecha(fecha):
    match = re.search(r'(\w+\s\d{1,2},\s\d{4})', fecha)
    if match:
        fecha_str = match.group(1)
        try:
            fecha_dt = pd.to_datetime(fecha_str)
            return fecha_dt.strftime('%Y-%m-%d')
        except:
            return 'Fecha inválida'
    else:
        return 'Formato inválido'


In [28]:
#Se crea un nuevo campo 'date'
df_reviuw['date'] = df_reviuw['posted'].apply(convertir_fecha)


In [30]:
#Se Elimina 'posted'
df_reviuw = df_reviuw.drop('posted', axis=1)


In [31]:
df_reviuw.head()

,funny,last_edited,item_id,helpful,recommend,user_id,user_url,sentiment_analisy,date
0,,,1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-11-05
1,,,22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-07-15
2,,,43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-04-21
3,,,251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,2,2014-06-24
4,,,227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,0,2013-09-08


In [32]:
# Se exporta un archivo csv del df_reviuw
df_reviuw.to_csv('d_reviuws.csv')

: 